In [2]:
%reload_ext autoreload
%autoreload 2

import time
from tqdm import tqdm
import numpy as np
import scipy as sp
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.transforms import v2 as transforms

from matplotlib import pyplot as plt

from networkAlignmentAnalysis.models.registry import get_model
from networkAlignmentAnalysis.datasets import get_dataset
from networkAlignmentAnalysis.experiments.registry import get_experiment
from networkAlignmentAnalysis import utils
from networkAlignmentAnalysis import files
from networkAlignmentAnalysis import train

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device: ', DEVICE)

using device:  cuda


In [3]:
# TODO
# 1.1. include additional AlignmentModel methods stored in extra class in base model

# 4. Rewrite existing analysis pipelines
# 5. SLURM!!!!

# Figure out why convolutional alignment measurement is slow...
# still working on if it's possible to speed up measure_alignment for convolutional layers

# Basic alignment_comparison Analyses (or maybe for alignment_stats):
# - compare initial to final alignment...
# - compare initial alignment to delta weight norm...
# - observe alignment of delta weight
# - compare alignment to outgoing delta weight norm!

# Eigenfeature analyses:
# done: - start by just looking at amplitude of activity on each eigenvector within each layer
# - Determine contribution of each eigenfeature on performance with a eigenvector dropout experiment
# - Measure beta_adversarial (figure out how adversarial examples map onto eigenvectors)

# alignmentShaping.ipynb has an adversarial experiment worth looking at

# Consider Valentin's idea about measuring an error threshold given signal and noise for a given level of alignment
# e.g. plot a 2d heatmap comparing the noise amplitude and the average alignment
# and then think about how to apply this to network design...

In [5]:
model_name = 'CNN2P2'
dataset_name = 'MNIST'

net = get_model(model_name, build=True, dataset=dataset_name).to(DEVICE)
dataset = get_dataset(dataset_name, build=True, transform_parameters=net, device=DEVICE)

optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
results = train.train([net], [optimizer], dataset, num_epochs=10, alignment=False)

beta, eigenvalue, eigenvector = net.measure_eigenfeatures(dataset.test_loader, by_stride=True)

100%|██████████| 10/10 [00:02<00:00,  4.62it/s]


In [3]:
# next steps:
# - finish eigenvector dropout methods -- need to do some linear algebra and decide how to renormalize
# - integrate into alignment_stats and observe results!!!
# --
# I'd like to make the conv handling code a bit smarter!
# And also more readable...
# Move some of the conv layer handling methods to internal AlignmentNetwork methods...
# Then integrate those methods into the 3 eigenfeature methods for easy reading and clear code
# --

In [ ]:
# places that use 'unfold' or should use it
# -- done(unless adding more support) -- AN.get_alignment_weights()
# AN.forward_eigenvector.dropout() # --- doesn't use it but it should!!! --- 
# -- done -- AN.measure_eigenfeatures()
# AN.measure_class_eigenfeatures()

In [14]:
model_name = 'CNN2P2'
dataset_name = 'MNIST'

prms = dict(
    channels=[32, 64],
    kernel_size=[5, 5],
    stride=[1, 1],
    padding=[2, 2],
    num_hidden=[3136, 128],
)

net = get_model(model_name, build=True, dataset=dataset_name, **prms).to(DEVICE)
# dataset = get_dataset(dataset_name, build=True, transform_parameters=net, device=DEVICE)

In [8]:
batch = next(iter(dataset.test_loader))
images, label = dataset.unwrap_batch(batch)

In [18]:
store = []
x = images.clone()
for layer in net.layers:
    print(x.shape)
    store.append(x)
    x = layer(x)
print(x.shape)
print([s.shape for s in store])

torch.Size([1024, 1, 28, 28])
torch.Size([1024, 32, 14, 14])
torch.Size([1024, 3136])
torch.Size([1024, 128])
torch.Size([1024, 10])
[torch.Size([1024, 1, 28, 28]), torch.Size([1024, 32, 14, 14]), torch.Size([1024, 3136]), torch.Size([1024, 128])]


In [21]:
layer = net.metaparameters[1]['layer_handle'](net.layers[1])
x = store[1]

subspace = None
by_stride = True

weight = layer.weight.data
layer_prms = utils.get_unfold_params(layer)
ux = torch.nn.functional.unfold(x, layer.kernel_size, **layer_prms)
if subspace:
    if by_stride:
        ux = torch.matmul(subspace, torch.matmul(subspace.T, ux))
    else:
        # need to project out subspace in "full" convolutional subspace
        # start by flattening unfolded input 
        ux = ux.transpose(1, 2).reshape(x.size(0), -1)
        
h_max, w_max = utils.get_maximum_strides(x.size(2), x.size(3), layer)
weight = weight.view(weight.size(0), -1)
out = torch.matmul(weight, ux).view(x.size(0), weight.size(0), h_max, w_max) + layer.bias.view(-1, 1, 1)


In [46]:
for idx, sublayer in enumerate(net.layers[1]):
    print(idx, sublayer)

0 Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
1 ReLU()
2 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
3 Flatten(start_dim=1, end_dim=-1)
